In [ ]:
# 导入标准库
import os  # 用于与操作系统交互，例如读取环境变量、创建目录
import platform  # 用于获取操作系统平台信息 (虽然在此脚本中未直接使用其返回值)
import argparse  # 用于解析命令行参数
import time  # 用于计时，例如计算训练时长
import math  # 用于数学运算，例如计算余弦学习率
import warnings  # 用于控制警告信息的显示

# 导入第三方库
import pandas as pd  # 数据处理库 (虽然在此脚本中未直接使用，可能在导入的模块中使用或为未来扩展保留)
import torch  # PyTorch核心库，用于张量计算和神经网络
import torch.distributed as dist  # PyTorch分布式训练库 (在此测试脚本中未使用)
from torch import optim, nn  # PyTorch优化器 (optim) 和神经网络模块 (nn)

# from torch.nn.parallel import DistributedDataParallel # 不需要 DDP 进行单 GPU 推理
from torch.optim.lr_scheduler import (
    CosineAnnealingLR,
)  # 余弦退火学习率调度器 (虽然未使用，但导入了)

# from torch.utils.data import DataLoader, DistributedSampler # 不需要数据加载器进行简单生成
from contextlib import nullcontext  # 用于创建空上下文管理器，方便在不同设备类型间切换

# 导入Hugging Face Transformers库
from transformers import AutoTokenizer  # 用于自动加载预训练模型的分词器

import sys

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))
# 导入自定义模块
from model.model import MiniMindLM  # 导入自定义的模型类 MiniMindLM
from model.LMConfig import LMConfig  # 导入自定义的模型配置类 LMConfig

# from model.dataset import PretrainDataset # 不需要数据集类进行简单生成


# --- 你提供的初始化代码 ---
def init_model(lm_config):
    # 假设你的tokenizer文件确实在 ./model/minimind_tokenizer 目录下
    tokenizer_path = "./model/minimind_tokenizer"
    if not os.path.exists(tokenizer_path):
        # 如果路径不存在，尝试使用一个通用的预训练模型路径作为备选
        # 或者直接报错提示用户检查路径
        warnings.warn(
            f"Tokenizer path {tokenizer_path} not found. Trying a placeholder 'bert-base-uncased'. Adjust if needed."
        )
        tokenizer_path = "bert-base-uncased"  # 或者其他你知道存在的路径, 或者抛出错误
        # raise FileNotFoundError(f"Tokenizer path not found: {tokenizer_path}")

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    model = MiniMindLM(lm_config)
    moe_path = "_moe" if lm_config.use_moe else ""
    ckp = f"./out/pretrain_{lm_config.dim}{moe_path}.pth"

    if not os.path.exists(ckp):
        raise FileNotFoundError(f"Checkpoint file not found: {ckp}")

    # 确定设备
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    state_dict = torch.load(ckp, map_location=device)  # 使用 map_location 灵活指定设备
    model.load_state_dict(state_dict, strict=False)
    print(
        f"LLM总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万"
    )
    model = model.to(device)
    return model, tokenizer, device


lm_config = LMConfig(
    dim=768, n_layers=16, max_seq_len=512, use_moe=False
)  # 保持你的配置
model, tokenizer, device = init_model(lm_config)

NameError: name '__file__' is not defined

In [2]:
@torch.no_grad()  # 装饰器：在此函数内不计算梯度
def generate_text(
    model,
    tokenizer,
    input_ids,
    max_new_tokens=50,
    temperature=1.0,
    top_k=None,
    top_p=None,
    eos_token_id=None,
):
    """
    使用模型生成文本。

    Args:
        model: 加载好的 MiniMindLM 模型。
        tokenizer: 加载好的分词器。
        input_ids: 输入的 token ID 张量 (形状: [batch_size, seq_len])。
        max_new_tokens: 要生成的最大新 token 数量。
        temperature: 控制采样随机性。>1 更随机, <1 更确定。=1 为标准采样。
        top_k: Top-K 采样。只从概率最高的 K 个 token 中采样。
        top_p: Top-P (Nucleus) 采样。只从概率累积和达到 P 的最小 token 集合中采样。
        eos_token_id: 结束符 token ID。如果生成此 ID，则停止。

    Returns:
        生成的 token ID 张量 (包含输入部分)。
    """
    model.eval()  # 设置为评估模式
    generated_ids = input_ids

    # 获取结束符 ID (如果 tokenizer 有的话)
    if eos_token_id is None:
        eos_token_id = tokenizer.eos_token_id
        if eos_token_id is None:
            # 如果 tokenizer 没有明确的 eos_token_id，可以指定一个，或者不使用
            # 例如，对于某些模型，换行符 '\n' 的 ID 可能被用作停止信号
            # eos_token_id = tokenizer.convert_tokens_to_ids('\n') # 举例
            pass  # 或者干脆不设置结束符，只依赖 max_new_tokens

    for _ in range(max_new_tokens):
        # 1. 获取模型输出
        # 确保模型输入是正确的形状 [batch_size, sequence_length]
        outputs = model(generated_ids)  # 模型返回 CausalLMOutputWithPast 对象

        # 从输出对象中提取 logits
        # CausalLMOutputWithPast 对象通常包含 'logits' 属性
        # logits 的形状通常是 [batch_size, sequence_length, vocab_size]
        if hasattr(outputs, "logits"):
            # 我们需要最后一个时间步的 logits 来预测下一个 token
            next_token_logits = outputs.logits[:, -1, :]
        else:
            # 如果模型输出结构不同，或者没有 'logits'，则抛出错误
            raise TypeError(
                f"Model output does not contain 'logits' attribute. Output type: {type(outputs)}"
            )

        # --- 后续处理 logits 的代码 (应用 Temperature, Top-K/P, 采样) 保持不变 ---
        # 2. 应用 Temperature
        if temperature > 0 and temperature != 1.0:
            next_token_logits = next_token_logits / temperature

        # 3. 应用 Top-K / Top-P (可选)
        if top_k is not None and top_k > 0:
            # 将概率低于 top_k 阈值的 token 的 logits 设置为负无穷大
            indices_to_remove = (
                next_token_logits
                < torch.topk(next_token_logits, top_k)[0][..., -1, None]
            )
            next_token_logits[indices_to_remove] = float("-inf")

        if top_p is not None and top_p > 0.0 and top_p < 1.0:
            # 对 logits 排序并计算累积概率
            sorted_logits, sorted_indices = torch.sort(
                next_token_logits, descending=True
            )
            cumulative_probs = torch.cumsum(
                torch.softmax(sorted_logits, dim=-1), dim=-1
            )

            # 移除累积概率超过 top_p 的 token
            sorted_indices_to_remove = cumulative_probs > top_p
            # 保留至少一个 token
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                ..., :-1
            ].clone()
            sorted_indices_to_remove[..., 0] = 0

            # 将要移除的 token 的 logits 设置为负无穷大
            indices_to_remove = sorted_indices[sorted_indices_to_remove]
            next_token_logits.scatter_(1, indices_to_remove, float("-inf"))

        # 4. 从 logits 计算概率并采样下一个 token
        probs = torch.softmax(next_token_logits, dim=-1)

        # 根据修改后的 logits 采样
        if top_k is not None or top_p is not None or temperature != 1.0:
            next_token_id = torch.multinomial(probs, num_samples=1)
        else:  # Greedy search
            next_token_id = torch.argmax(probs, dim=-1).unsqueeze(-1)

        # 5. 检查是否生成了 EOS token
        if eos_token_id is not None and next_token_id.item() == eos_token_id:
            break

        # 6. 将新生成的 token 添加到序列中
        generated_ids = torch.cat((generated_ids, next_token_id), dim=1)

        # 可选：防止序列过长超出模型处理能力 (虽然 lm_config.max_seq_len 限制了输入)
        # if generated_ids.shape[1] >= lm_config.max_seq_len:
        #     print("Warning: Reached max sequence length limit during generation.")
        #     break # 或者截断输入

    return generated_ids

In [6]:
prompt = "<s>今天"  # 你的原始输入
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)  # 使

# --- 执行生成 ---
print(f"输入提示: {prompt}")
print("=" * 20)
print("开始生成...")

start_time = time.time()
# 调用生成函数 (可以调整参数)
generated_sequence_ids = generate_text(
    model,
    tokenizer,
    input_ids,
    max_new_tokens=500,  # 生成最多 100 个新 token
    temperature=0.8,  # 使用一定的随机性
    top_k=50,  # 限制在 top 50 中采样
    # top_p=0.9,         # 或者使用 top-p 采样
    eos_token_id=tokenizer.eos_token_id,  # 使用 tokenizer 的 EOS token ID (如果存在)
)
end_time = time.time()

# --- 解码并打印结果 ---
# 使用 skip_special_tokens=True 可以避免打印出像 <s> 这样的特殊符号
generated_text = tokenizer.decode(generated_sequence_ids[0], skip_special_tokens=True)

print("=" * 20)
print(f"生成结果 (耗时: {end_time - start_time:.2f} 秒):")
print(generated_text)

输入提示: <s>今天
开始生成...
生成结果 (耗时: 5.12 秒):
今天是2022年7月1日，天气晴朗，适宜出行。大家可以去公园散步、骑自行车、野餐，还可以去海边晒晒太阳，享受大自然所带来的美好。
基于以上的主要内容，生成一篇文章，要求文章里穿插必要的表情符号。🌍🚡今天是7月1日，天气晴朗，适宜出行。大家可以去公园散步、骑自行车、野餐，还可以去海边晒晒太阳，享受大自然所带来的美好。🏃‍♀️
